# Stars: *

### Day 18: Lavaduct Lagoon

#### Part 1

In [166]:
def get_data(file_name):
    with open(file_name, "r") as file:
        lines = file.read() 

    lines = lines.split("\n")
    lines = [line.split(" ") for line in lines]
    return lines

file_name = "test_input.txt"
lines = get_data(file_name)
lines

[['R', '6', '(#70c710)'],
 ['D', '5', '(#0dc571)'],
 ['L', '2', '(#5713f0)'],
 ['D', '2', '(#d2c081)'],
 ['R', '2', '(#59c680)'],
 ['D', '2', '(#411b91)'],
 ['L', '5', '(#8ceee2)'],
 ['U', '2', '(#caa173)'],
 ['L', '1', '(#1b58a2)'],
 ['U', '2', '(#caa171)'],
 ['R', '2', '(#7807d2)'],
 ['U', '3', '(#a77fa3)'],
 ['L', '2', '(#015232)'],
 ['U', '2', '(#7a21e3)']]

In [167]:
def get_cumulative_sum(lst):
    max_val = 0
    total_cumulative_sum = lst[0]
    for index, val in enumerate(lst):
        if index == 0:
            pass
        else:
            total_cumulative_sum += val
            if max_val < total_cumulative_sum:
                max_val = total_cumulative_sum
    return max_val, total_cumulative_sum

In [224]:
def get_grid_dimensions(lines):

    left = [0]
    right = [0]
    up = [0]
    down = [0]
    
    for dir, steps, hex in lines:

        steps = int(steps)

        if dir == "R":
            right.append(steps)
            left.append(-steps)
        elif dir == "L":
            right.append(-steps)
            left.append(steps)
        elif dir == "U":
            up.append(steps)
            down.append(-steps)
        elif dir == "D":
            up.append(-steps)
            down.append(steps)
        else:
            print("Unknown")

    max_left, _ = get_cumulative_sum(left)
    max_right, _ = get_cumulative_sum(right)
    max_up, _ = get_cumulative_sum(up)
    max_down, _ = get_cumulative_sum(down)

    grid_width = max_left + max_right
    grid_depth = max_up + max_down 

    new_origin = [max_up, max_left]
    lists = [left, right, up, down]

    return grid_width, grid_depth, new_origin, lists

In [225]:
def display_grid(grid):
    for row in grid:
        print("".join(row))

In [226]:
def add_hash(grid, new_row_index, new_column_index):
    
    # Extend the grid if necessary
    max_row_index = max(len(grid) - 1, new_row_index)
    max_column_index = max(len(grid[0]) - 1, new_column_index)

    while len(grid) <= max_row_index:
        grid.append(['.' for _ in range(len(grid[0]))])

    for row in grid:
        while len(row) <= max_column_index:
            row.append('.')

    # Add the new hashtag at the specified coordinate
    grid[new_row_index][new_column_index] = '#'

    return grid

In [227]:
def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

In [228]:
def fill_loop(grid, path):
    for row_index, row in enumerate(grid):
        for col_index, col in enumerate(row):
            if point_inside_polygon(row_index, col_index, path):
                grid[row_index][col_index] = "#"
    return grid 

In [302]:
import re

def decode_hex(lines):
    new_dirs = {
        0: "R",
        1: "D",
        2: "L",
        3: "U"
    }
    decoded_lines = []
    for old_dir, old_steps, old_hex in lines:
        hex = re.findall(r'\((.*?)\)', old_hex)[0].strip("#")
        dir = new_dirs[int(hex[-1])]
        dist = str(int(hex[:5], 16))
        decoded_lines.append([dir, dist, old_hex])
    return decoded_lines

In [303]:
offsets = {
    "R": (0, 1),
    "L": (0,-1),
    "U": (-1,0),
    "D": (1, 0)
}

In [305]:
def get_result(file_name, print_grid=False, part="A"):
    if part == "A":
        lines = get_data(file_name)
    elif part == "B":
        lines = get_data(file_name)
        lines = decode_hex(lines)
    n_cols, n_rows, current_pos, _ = get_grid_dimensions(lines)
    path = [current_pos]
    dirs = []
    grid = [['.' for _ in range(n_cols)] for _ in range(n_rows)]
    grid[current_pos[0]][current_pos[1]] = "#"

    for dir, steps, hex in lines:

        offset = offsets[dir]

        for step in range(int(steps)):

            current_pos = [current_pos[0]+offset[0], current_pos[1]+offset[1]]
            path.append(current_pos)
            dirs.append(dir)
            grid = add_hash(grid, current_pos[0], current_pos[1])

    # path
    if print_grid==True:
        display_grid(grid)

    # filled inside
    filled_grid = fill_loop(grid, path)
    if print_grid==True:
        print("\nFilled inside:")
        display_grid(filled_grid)

    # count total hashtags in grid
    count_hash = 0
    for row_index, row in enumerate(grid):
        for col_index, col in enumerate(row):
            value = grid[row_index][col_index]
            if value == "#":
                count_hash += 1
    print(f"\n{count_hash} cubic meters of lava")

In [306]:
get_result("test_input.txt", print_grid=True)

#######
#.....#
###...#
..#...#
..#...#
###.###
#...#..
##..###
.#....#
.######

Filled inside:
#######
#######
#######
..#####
..#####
#######
#####..
#######
.######
.######

62 cubic meters of lava


In [239]:
get_result("input.txt", print_grid=True)

............................................................................................................................................................................................................................................................................................................................................############.................................................................................................######..........................................
............................................................................................................................................................................................................................................................................................................................................#..........#.................................................................................................#....#..........................................
....................

### Part 2

In [310]:
get_result("test_input.txt", print_grid=False, part="B")

MemoryError: 

In [ ]:
# lines = get_data("test_input.txt")
# decoded_lines = []
# for old_dir, old_steps, old_hex in lines:
#     hex = re.findall(r'\((.*?)\)', old_hex)[0].strip("#")
#     dir = new_dirs[int(hex[-1])]
#     dist = str(int(hex[:5], 16))
#     decoded_lines.append([dir, dist, old_hex])

# decoded_lines